# 01 Data Preparation

Load and clean the OSM-derived grid data, then package it for remote runs.


In [35]:
from pathlib import Path
import sys
import os
import pypsa
import numpy as np

def find_repo_root(max_up=6):
    p = Path.cwd().resolve()
    for _ in range(max_up):
        if (p / 'README.md').exists() or (p / '.git').exists():
            return p
        if p.parent == p:
            break
        p = p.parent
    return Path.cwd().resolve()

repo_root = find_repo_root()
src_path = repo_root / 'src/'
if str(src_path) not in sys.path:
    sys.path.insert(1, str(src_path))
print(f"Using src path: {src_path}")
print(f"Repository root: {repo_root}")

import pypsa_simplified as ps

Using src path: /Users/jedrek/Documents/Studium Volkswirschaftslehre/3. Semester/European Energy Policy/HA/PyPSA---Simplified-European-Model/PyPSA---Simplified-European-Model/src
Repository root: /Users/jedrek/Documents/Studium Volkswirschaftslehre/3. Semester/European Energy Policy/HA/PyPSA---Simplified-European-Model/PyPSA---Simplified-European-Model


## Data sources and parsing
- **OSM prebuilt electricity network** (`data/raw/OSM Prebuilt Electricity Network/`): buses, lines, links, converters, transformers.
- **Custom CSV parsing**: `prepare_osm_source` uses a geometry-safe loader (handles commas inside WKT) to keep column counts correct.
- **Endpoint extraction**: First/last coordinates are pulled from WKT to map line/link endpoints to buses (tolerance 1e-5 degrees).
- **Country filter**: Defaults to DE/FR/PL/AT/IT; adjust via `countries` if needed.


In [36]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")
import pandas as pd
from pypsa_simplified import prepare_osm_source

osm_dir = repo_root / "data" / "raw" / "OSM Prebuilt Electricity Network"
sources = prepare_osm_source(osm_dir)
print({k: v.shape if hasattr(v, 'shape') else v for k, v in sources.items()})

OSMData = ps.data_prep.OSMData(sources)

{'buses': (6737, 10), 'lines': (8994, 16), 'converters': (67, 7), 'links': (38, 10), 'transformers': (875, 8), 'generators': None, 'loads': None, 'storage': None}


## Serialize for remote processing
The serialized artifact is compact (gzip + pickle) and ready to `scp` to the server.


In [37]:
# Save osm data source inputs
written = OSMData.save(output_path=repo_root / "data" / "processed" / "osm_source_data.pkl.gz")
print(f"OSMData.save wrote: {written}")

OSMData.save wrote: /Users/jedrek/Documents/Studium Volkswirschaftslehre/3. Semester/European Energy Policy/HA/PyPSA---Simplified-European-Model/PyPSA---Simplified-European-Model/data/processed/osm_source_data.pkl.gz


In [38]:
OSMData = ps.data_prep.OSMData(None)
load_path = repo_root / "data" / "processed" / "osm_source_data.pkl.pkl.gz"
OSMData.load(input_path=written)
print(f"Loaded OSMData from: {written}")

Loaded OSMData from: /Users/jedrek/Documents/Studium Volkswirschaftslehre/3. Semester/European Energy Policy/HA/PyPSA---Simplified-European-Model/PyPSA---Simplified-European-Model/data/processed/osm_source_data.pkl.gz


## Create a network
Use the OSMdata to create the first network

In [39]:
snapshots = pd.date_range("2024-01-01", "2024-12-31 23:00", freq="h")
NETWORK_METADATA = {
    'name': 'Simplified European Electricity Network',
    'snapshots': snapshots,
}

In [61]:
OSMData.get('lines')

,line_id,bus0,bus1,voltage,i_nom,circuits,s_nom,r,x,b,length,underground,under_construction,type,tags,geometry
0,merged_relation/10264161-225+1,way/150329778-225,way/239609000-225,225,1.29,1,502.728,0.355354,1.782691,2.326e-05,5922.56,f,f,Al/St 240/40 2-bundle 220.0,way/393286545-225;relation/10264161-225,'LINESTRING (5.972758751828162 43.157864156052...
1,merged_relation/11762422-400-b+2,way/761507281-400,way/111162936-400,400,2.58,1,1787.476,1.777477,14.575313,0.00025687,59249.24,f,f,Al/St 240/40 4-bundle 380.0,relation/11762422-400-b;relation/11762422-400-...,'LINESTRING (6.0555323486868655 46.25095295760...
2,merged_relation/1208415-380-c+1,way/23380800-380,way/24478889-380,380,2.58,1,1698.103,0.281152,2.30545,4.063e-05,9371.75,f,f,Al/St 240/40 4-bundle 380.0,relation/1208415-380-c;relation/1208415-380-b,'LINESTRING (6.617347419706488 51.475937777129...
3,merged_relation/14096017-380-b+3,way/936521586-380,way/234983117-380,380,2.58,1,1698.103,0.544892,4.468113,7.874e-05,18163.06,f,f,Al/St 240/40 4-bundle 380.0,relation/14096017-380-b;relation/14096017-380-...,'LINESTRING (7.061264358931231 46.110634293306...
4,merged_relation/14096018-380-b+5,way/111162936-380,way/936521586-380,380,2.58,1,1698.103,2.33241,19.12576,0.00033706,77746.99,f,f,Al/St 240/40 4-bundle 380.0,relation/14096018-380-e;relation/14096018-380-...,'LINESTRING (6.61415579941978 46.5571996202857...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8989,way/99761296-380,AT13-380,AT12-380,380,2.58,2,3396.205,0.221959,1.820064,0.0001283,14797.27,f,f,Al/St 240/40 4-bundle 380.0,way/99761296-380,"'LINESTRING (15.9900061 47.25702409221586, 15...."
8990,way/997803612-330,LT11-330,LT12-330,330,1.935,1,1106.001,0.06908,0.457652,7.16e-06,1726.99,f,f,Al/St 240/40 3-bundle 300.0,way/997803612-330,'LINESTRING (24.624891444927925 54.76142039863...
8991,way/997803613-330,LT12-330,LT11-330,330,1.935,2,2212.002,0.033948,0.224906,1.408e-05,1697.4,f,f,Al/St 240/40 3-bundle 300.0,way/997803613-330,'LINESTRING (24.624891444927925 54.76142039863...
8992,way/998005837-1-380,DE23-380,DE24-380,380,2.58,1,1698.103,0.038156,0.312882,5.51e-06,1271.88,f,f,Al/St 240/40 4-bundle 380.0,way/998005837-1-380,'LINESTRING (13.515015780259757 52.61950084610...


In [ ]:
print(sum(OSMData.get('lines')['circuits'].isna()))
set(OSMData.get('lines')['circuits'])

{'1', '2', '3', '4'}

In [ ]:


n = pypsa.Network()
n = ps.build_network_from_source(n, OSMData, None)



ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

### Next
- Use `notebooks/main.ipynb` to transfer the artifact and trigger the remote optimization.
- For custom country lists or tolerance, pass `countries`/`tol` to `prepare_osm_source`.
